In [ ]:
from rosetta.protocols.rosetta_scripts import *
from pyrosetta import *
from pyrosetta import PyMOLMover
from rosetta.core.pack.task import *
init()

In [ ]:
pose = pose_from_pdb("truncated_fkb.pdb")

In [ ]:
working_pose = pose.clone()

In [ ]:
scorefxn = create_score_function("ref2015_cart")

MoveMap

In [ ]:
# For a specific refion of residues
res_action = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector()
res_action.set_focus(str(47))
res_action.set_distance(12)
res_action.set_include_focus_in_subset(True) 

In [ ]:
# For a specific array of residues

#res_select = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector()
#res_select.set_index_range(1,10)

In [ ]:
movemap = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
movemap.all_jumps(False)
movemap.all_bb(False)
movemap.add_bb_action(pyrosetta.rosetta.core.select.movemap.mm_enable, res_action)
movemap.add_chi_action(pyrosetta.rosetta.core.select.movemap.mm_enable, res_action)
movemap.set_cartesian(True) 

In [ ]:
tf = TaskFactory()
tf.push_back(operation.InitializeFromCommandline())
tf.push_back(operation.RestrictToRepacking())

prevent_repacking_rlt = operation.PreventRepackingRLT()
prevent_subset_repacking = operation.OperateOnResidueSubset(prevent_repacking_rlt, res_action, True)
tf.push_back(prevent_subset_repacking)

packer = pyrosetta.rosetta.protocols.minimization_packing.PackRotamersMover()
packer.score_function(scorefxn)
packer.task_factory(tf)

fastrelax = pyrosetta.rosetta.protocols.relax.FastRelax()
fastrelax.set_scorefxn(scorefxn)
fastrelax.cartesian(True)
fastrelax.set_movemap_factory(movemap)
fastrelax.set_task_factory(tf)

fastrelax.apply(working_pose) 

In [ ]:
working_pose.dump_pdb("relaxed_47.pdb")